In [1]:
import glob
import pandas as pd
import os
from xml.etree import ElementTree as ET
from typing import List
from sklearn.model_selection import train_test_split

In [36]:
def convert_annots(annotations: List[str], image_dir: str):
    """
    Function to convert xml of annotations to a dataframe for easier processing
    Args:
        annots: a list of string file paths
        image_dir: path to the image folder
        
    Returns: a daframe containing xy, height, width, and label class
    """
    data = {
        'image_path': [],
        'xmin': [],
        'ymin': [],
        'xmax': [],
        'ymax': []
    }
    labels = []
    
    for annot in annotations:
        tree = ET.parse(annot)
        root = tree.getroot()
        file_name = root.find('filename').text
        image_path = glob.glob(os.path.join(image_dir, file_name))[0]
        
        for obj in root.findall('object'):
            data['image_path'].append(image_path)
            labels.append(obj.find('name').text)
            for bb in obj.find('bndbox'):
                data[bb.tag].append(int(bb.text))
    
    print(labels)
    return train_test_split(pd.DataFrame(data), labels, test_size=0.2, random_state=2021)

In [11]:
def get_annots(annot_path: str, image_dir: str) -> pd.DataFrame:
    """
    Converts string of images 
    """
    annotations = glob.glob(os.path.join(annot_path, '*.xml'))
    return convert_annots(annotations, image_dir)

In [37]:
get_annots('../data/archive/train_zip/train', '../data/archive/train_zip/train')[1]

['orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange', 'banana', 'banana', 'banana', 'banana', 'banana', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'banana', 'orange', 'orange', 'banana', 'banana', 'banana', 'apple', 'apple', 'apple', 'banana', 'banana', 'orange', 'orange', 'orange', 'orange', 'banana', 'banana', 'banana', 'banana', 'apple', 'apple', 'orange', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'orange', 'apple', 'orange', 'orange', 'apple', 'apple', 'orange', 'orange', 'orange', 'orange', 'apple', 'banana', 'orange', 'banana', 'apple', 'banana', 'banana', 'orange', 'orange', 'apple', 'banana', 'orange', 'orange', 'banana', 'orange', 'apple', 'banana', 'banana', 'banana', 'orange', 'orange', 'orange', 'apple', 'apple', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'banana', 'orange', 'orange', 'orange', 'orange', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'apple', 'banana', 'banana', 'banana', 'ba

,image_path,xmin,ymin,xmax,ymax
247,../data/archive/train_zip/train/apple_2.jpg,37,52,205,222
124,../data/archive/train_zip/train/orange_3.jpg,97,1,203,115
34,../data/archive/train_zip/train/banana_30.jpg,108,91,430,228
297,../data/archive/train_zip/train/apple_65.jpg,333,239,574,463
179,../data/archive/train_zip/train/banana_66.jpg,172,67,395,282
...,...,...,...,...,...
228,../data/archive/train_zip/train/orange_29.jpg,76,69,412,439
338,../data/archive/train_zip/train/banana_51.jpg,186,39,624,382
319,../data/archive/train_zip/train/orange_40.jpg,244,202,663,600
377,../data/archive/train_zip/train/banana_6.jpg,889,147,1461,1147


In [18]:
df

,image_path,xmin,ymin,xmax,ymax
